In [1]:
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os

In [5]:

"""
some variables needed to be set up by users
"""
conference = 'icml'
conference_url = "http://proceedings.mlr.press/v139/" # the conference url to download papers from
edgedriver_path = 'C:\\Users\\xiaoyang\\Downloads\\edgedriver_win64\msedgedriver.exe' # the chromedriver.exe path
root = r'E:\OneDrive\academics\papers\conferences\ICML-2021-ALL'.replace('\\','/') # file path to save the downloaded papers

os.makedirs(root, exist_ok=True)
print(root)

E:/OneDrive/academics/papers/conferences/ICML-2021-ALL


In [6]:
"""
conference urls examples

cvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)
eccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)
eccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)
iccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)
icml: http://proceedings.mlr.press/v119/ (ICML 2020)
icml: http://proceedings.mlr.press/v139/ (ICML 2021)
neurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)
iclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)
siggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)

"""




'\nconference urls examples\n\ncvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)\neccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)\neccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)\niccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)\nicml: http://proceedings.mlr.press/v119/ (ICML 2020)\nneurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)\niclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)\nsiggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)\n\n'

In [7]:
driver = webdriver.Edge(edgedriver_path)  
driver.get(conference_url)

from retrieve_titles_urls_from_websites import *
retrieve = globals()['retrieve_from_'+conference]
print('Retrieving pdf urls. This could take some time...')
pdfurllist, pdfnamelist = retrieve(driver)



Retrieving pdf urls. This could take some time...


In [8]:
# check the retrieved urls
print('The first 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[i])
print('\nThe last 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[-(i+1)])


The first 5 pdf urls:

http://proceedings.mlr.press/v139/abdolshah21a/abdolshah21a.pdf
http://proceedings.mlr.press/v139/abeyrathna21a/abeyrathna21a.pdf
http://proceedings.mlr.press/v139/acar21a/acar21a.pdf
http://proceedings.mlr.press/v139/acar21b/acar21b.pdf
http://proceedings.mlr.press/v139/acharya21a/acharya21a.pdf

The last 5 pdf urls:

http://proceedings.mlr.press/v139/zweig21a/zweig21a.pdf
http://proceedings.mlr.press/v139/zou21b/zou21b.pdf
http://proceedings.mlr.press/v139/zou21a/zou21a.pdf
http://proceedings.mlr.press/v139/zintgraf21a/zintgraf21a.pdf
http://proceedings.mlr.press/v139/zimmermann21a/zimmermann21a.pdf


In [9]:
# check the retrieved paper titles
print('The first 5 pdf titles:\n')
for i in range(5):
    
    print(pdfnamelist[i])
print('\nThe last 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[-(i+1)])


The first 5 pdf titles:

A New Representation of Successor Features for Transfer across Dissimilar Environments
Massively Parallel and Asynchronous Tsetlin Machine Architecture Supporting Almost Constant-Time Scaling
Debiasing Model Updates for Improving Personalized Federated Training
Memory Efficient Online Meta Learning
Robust Testing and Estimation under Manipulation Attacks

The last 5 pdf titles:

A Functional Perspective on Learning Symmetric Functions with Neural Networks
On the Convergence of Hamiltonian Monte Carlo with Stochastic Gradients
Provable Robustness of Adversarial Training for Learning Halfspaces with Noise
Exploration in Approximate Hyper-State Space for Meta Reinforcement Learning
Contrastive Learning Inverts the Data Generating Process


In [10]:
print('The number of papers is ', len(pdfnamelist))
assert len(pdfnamelist)==len(pdfurllist), 'The number of titles and the number of urls are not matched. \
                                            You might solve the problem by checking the HTML code in the \
                                            website yourself or you could ask the author by raising an issue.'

The number of papers is  1183


In [11]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
for i, url in enumerate(pdfurllist):
    if url != None :      
        pdfname = slugify(pdfnamelist[i])
        if os.path.isfile(root+'/'+pdfname+".pdf"):
            print('existed', i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
        else:
            print(i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
            
            try_download = True
            while try_download:
                try:
                    data = requests.get(pdfurllist[i], timeout=80, headers=headers).content
                    try_download = False
                except ConnectionError as e: 
                    _ = time.sleep(random.uniform(1,2)) # for anti-anti-crawler purpose
                    
            with open(root+'/'+pdfname+".pdf", 'wb')  as f:
                f.write(data)  

Start downloading
0 	 A New Representation of Successor Features for Transfer across Dissimilar Environments 	 http://proceedings.mlr.press/v139/abdolshah21a/abdolshah21a.pdf
1 	 Massively Parallel and Asynchronous Tsetlin Machine Architecture Supporting Almost Constant-Time Scaling 	 http://proceedings.mlr.press/v139/abeyrathna21a/abeyrathna21a.pdf
2 	 Debiasing Model Updates for Improving Personalized Federated Training 	 http://proceedings.mlr.press/v139/acar21a/acar21a.pdf
3 	 Memory Efficient Online Meta Learning 	 http://proceedings.mlr.press/v139/acar21b/acar21b.pdf
4 	 Robust Testing and Estimation under Manipulation Attacks 	 http://proceedings.mlr.press/v139/acharya21a/acharya21a.pdf
5 	 GP-Tree: A Gaussian Process Classifier for Few-Shot Incremental Learning 	 http://proceedings.mlr.press/v139/achituve21a/achituve21a.pdf
6 	 f-Domain Adversarial Learning: Theory and Algorithms 	 http://proceedings.mlr.press/v139/acuna21a/acuna21a.pdf
7 	 Towards Rigorous Interpretations: a F